# CIFAR-100 Prediction - Evaluation

This notebook is used to check the results and trained models and to test their quality.

## Data Loading

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import json

In [ ]:
classes = tf.io.read_file("../data/raw/classes.txt")
classes = tf.strings.split(classes, "\n")

## Model Evaluation

### Convolutional Neural Network

In [ ]:
cnn_model = keras.saving.load_model("../models/cnn_model.keras")
cnn_history = np.load("../models/cnn_history.npy", allow_pickle=True).item()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(1, 1, 1)

num_epochs = len(cnn_history["accuracy"])
num_ticks = min(10, num_epochs)
x_ticks = np.linspace(1, num_epochs, num_ticks, dtype=int)

ax.plot(x_ticks, cnn_history["accuracy"], label="accuracy")
ax.plot(x_ticks, cnn_history["val_accuracy"], label="val_accuracy")
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy")
ax.set_xticks(x_ticks)
ax.legend()

plt.show()

In [ ]:
with open("../dvclive/cnn/evaluation/metrics.json", "r") as file:
    cnn_metrics = json.load(file)

print(f"Train accuracy: {round(cnn_metrics['train']['accuracy'], 4)}")
print(f"Test accuracy: {round(cnn_metrics['test']['accuracy'], 4)}")

## Model Prediction

In [ ]:
final_model = cnn_model

In [ ]:
test_dataset = (tf.data.Dataset.load("../data/preprocessed/test/cnn")
    .batch(32)
    .prefetch(tf.data.AUTOTUNE))

In [ ]:
fig = plt.figure(figsize=(10, 10))
batch_images, batch_classes = next(iter(test_dataset.take(1)))
predictions = final_model.predict(batch_images)

for index in range(25):
    ax = plt.subplot(5, 5, index + 1)

    predicted_class = np.argmax(predictions[index])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(batch_images[index])
    ax.set_xlabel(f"{classes[predicted_class]} ({round(float(predictions[index][predicted_class]), 3)})")

plt.show()